# Spectrum

In [1]:
import pandas as pd
import astropy.units as u
import numpy as np
import re
import matplotlib.pyplot as plt
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from pandas.plotting import scatter_matrix
from definitions import *

In [2]:
catalog = pd.read_csv("markeddata/" + get_name_for_gevtev() + ".txt", sep='\t', index_col=0)
catalog.shape

(141, 170)

In [3]:
xmm_catalog = pd.read_csv("markeddata/" + get_name_for_gevtevxmm() + ".txt", sep='\t', index_col=0)
xmm_catalog.shape

(2175, 215)

In [4]:
xmm_catalog[list_xmm_spectra_columns()]

,xmm_SC_EP_1_FLUX,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX
0,7.841600e-13,1.386490e-11,3.208440e-11,3.098090e-11,1.091600e-11
1,6.025060e-13,1.177800e-11,2.542440e-11,2.179990e-11,5.282680e-12
2,5.859880e-15,1.510420e-13,1.740440e-13,8.414880e-14,0.000000e+00
3,2.615900e-14,4.028110e-13,6.947540e-13,5.540100e-13,4.957800e-15
4,1.920070e-12,3.786540e-11,7.193580e-11,6.468450e-11,1.292160e-11
5,1.535850e-14,2.944520e-13,5.891280e-13,4.612800e-13,1.984000e-15
6,2.546390e-13,5.510890e-12,1.139300e-11,5.809570e-12,7.014450e-13
7,1.237660e-14,1.100390e-14,5.605320e-13,6.895040e-13,3.835640e-14
8,8.790840e-15,2.218820e-13,2.404500e-13,2.110470e-13,1.029240e-14
9,5.414970e-15,3.951300e-14,4.277870e-13,5.064290e-13,6.886070e-15


In [5]:
# columns contain spectrums in xmm
xmm_spectrum_columns = list_xmm_spectra_columns()

In [6]:
catalog = catalog.join(xmm_catalog.loc[catalog.index, xmm_spectrum_columns])
catalog

,index,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_where,s_MAIN_ID,s_class,s_simbad,marked,xmm_SC_EP_1_FLUX,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX
0,33,b' ',b' ',b' ',b' ',b'RBS 0030 ',b' ',b' ',b' ',b' ',...,b'egal',[],[],0,0,7.841600e-13,1.386490e-11,3.208440e-11,3.098090e-11,1.091600e-11
1,59,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,b'gal',[],[],0,0,6.025060e-13,1.177800e-11,2.542440e-11,2.179990e-11,5.282680e-12
2,75,b'0FGL J0033.6-1921',b'1FGL J0033.5-1921 ',b'1FHL J0033.6-1921 ',b'2FGL J0033.5-1921 ',b'KUV 00311-1938 ',b' ',b' ',b' ',b' ',...,b'egal',[],[],0,0,5.859880e-15,1.510420e-13,1.740440e-13,8.414880e-14,0.000000e+00
3,78,b'0FGL J0036.7+5951',b'1FGL J0035.9+5951 ',b'1FHL J0035.9+5950 ',b'2FGL J0035.8+5951 ',b'1ES 0033+595 ',b' ',b' ',b' ',b' ',...,b'egal',"[""b'LS I +59 48'""]",['B'],1,0,2.615900e-14,4.028110e-13,6.947540e-13,5.540100e-13,4.957800e-15
4,99,b' ',b'1FGL J0047.3-2512 ',b' ',b'2FGL J0047.0-2516 ',b'NGC 253 ',b' ',b' ',b' ',b' ',...,b'egal',[],[],0,0,1.920070e-12,3.786540e-11,7.193580e-11,6.468450e-11,1.292160e-11
5,193,b'0FGL J0136.6+3903',b'1FGL J0136.5+3905 ',b'1FHL J0136.5+3906 ',b'2FGL J0136.5+3905 ',b'B3 0133+388 ',b' ',b' ',b' ',b' ',...,b'egal',[],[],0,0,1.535850e-14,2.944520e-13,5.891280e-13,4.612800e-13,1.984000e-15
6,214,b' ',b' ',b'1FHL J0152.6+0148 ',b'2FGL J0152.6+0148 ',b'PMN J0152+0146 ',b' ',b' ',b' ',b' ',...,b'egal',[],[],0,0,2.546390e-13,5.510890e-12,1.139300e-11,5.809570e-12,7.014450e-13
7,241,b' ',b'1FGL J0205.6+6449 ',b'1FHL J0205.7+6448 ',b'2FGL J0205.8+6448 ',b'PSR J0205+6449 ',b' ',b' ',b' ',b' ',...,b'gal',[],[],0,0,1.237660e-14,1.100390e-14,5.605320e-13,6.895040e-13,3.835640e-14
8,273,b'0FGL J0220.9+3607',b'1FGL J0221.0+3555 ',b'1FHL J0221.1+3555 ',b'2FGL J0221.0+3555 ',b'B0218+357 ',b' ',b' ',b' ',b' ',...,b'egal',[],[],0,0,8.790840e-15,2.218820e-13,2.404500e-13,2.110470e-13,1.029240e-14
9,278,b'0FGL J0222.6+4302',b'1FGL J0222.6+4302 ',b'1FHL J0222.6+4302 ',b'2FGL J0222.6+4302 ',b'3C 66A ',b' ',b' ',b'3EG J0222+4253',b'EGR J0223+4300 ',...,b'egal',[],[],0,0,5.414970e-15,3.951300e-14,4.277870e-13,5.064290e-13,6.886070e-15


Columns contained spectrum in gev:

In [7]:
gev_spectrum_columns = list_gev_spectrum_columns()

Columns contained spectrum in simbad:

In [8]:
s_spectrum_columns = list_s_spectrum_columns()

Columns contained spectrum in tev:

In [9]:
tev_spectrum_columns = list_tev_spectrum_columns()

In [10]:
s_other_columns = list_s_other_columns()

In [11]:
gev_other_columns = list_gev_other_columns()

In [12]:
tev_other_columns = list_tev_other_columns()

In [13]:
xmm_other_columns = list_xmm_other_columns()

In [14]:
spectrum_columns = gev_spectrum_columns + xmm_spectrum_columns + tev_spectrum_columns
spectrum_with_xmm_columns = spectrum_columns + xmm_spectrum_columns
other_columns = s_other_columns + gev_other_columns + tev_other_columns + ['marked']

In [15]:
spectrums = catalog[spectrum_columns + other_columns].copy()

The next cell computes simbad spectrum using this formula:
$$ F_{simbad} = 10^{-flux_{simbad}}$$

In [16]:
#spectrums[s_spectrum_columns] = 10 ** (-spectrums[s_spectrum_columns])

The next cell computes TeV spectrum using:
$$F_{TeV} = 1.6*E^{2}*Flux$$

In [17]:
#for i in tev_spectrum_columns:
#    if len(re.findall("\d+", i)) == 2:
#        E = 0.3
#    else:
#        E = int(re.findall("\d+", i)[0])
#    spectrums[i] = 1.6 * E ** 2 * spectrums[i]

In [18]:
spectrums = spectrums.fillna(0)
spectrums = np.log(spectrums[spectrum_columns] + 1)
# spectrums = np.log(spectrums[spectrum_columns])
# spectrums[(spectrums == -np.inf)] = 0

In [19]:
(spectrums == -np.inf).any()

gev_nuFnu10000_100000    False
gev_nuFnu1000_3000       False
gev_nuFnu100_300         False
gev_nuFnu3000_10000      False
gev_nuFnu300_1000        False
gev_nuFnu30_100          False
xmm_SC_EP_1_FLUX         False
xmm_SC_EP_2_FLUX         False
xmm_SC_EP_3_FLUX         False
xmm_SC_EP_4_FLUX         False
xmm_SC_EP_5_FLUX         False
tev_0.3TeV               False
tev_1TeV                 False
tev_3TeV                 False
tev_10TeV                False
tev_30TeV                False
dtype: bool

In [20]:
display(spectrums)

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,xmm_SC_EP_1_FLUX,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV
0,1.474376e-13,5.384582e-13,0.000000e+00,6.283862e-13,7.549517e-14,0.0,7.842615e-13,1.386491e-11,3.208434e-11,3.098100e-11,1.091593e-11,1.046718e-12,1.947331e-13,3.530509e-14,5.329071e-15,8.881784e-16
1,1.018519e-12,1.078249e-12,5.018208e-14,5.646594e-13,2.145617e-12,0.0,6.024070e-13,1.177791e-11,2.542433e-11,2.179990e-11,5.282663e-12,1.975531e-12,3.010925e-13,9.503509e-14,8.750778e-13,6.631362e-12
2,8.395062e-12,4.175993e-12,2.267297e-12,6.302514e-12,2.296607e-12,0.0,5.773160e-15,1.509903e-13,1.740830e-13,8.415491e-14,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,5.756728e-12,3.369749e-12,2.990053e-12,4.720002e-12,4.563461e-12,0.0,2.620126e-14,4.027889e-13,6.947776e-13,5.540013e-13,4.884981e-15,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,5.417888e-13,1.216804e-12,2.193579e-12,1.207701e-12,1.687095e-12,0.0,1.920020e-12,3.786549e-11,7.193579e-11,6.468448e-11,1.292166e-11,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
5,1.884848e-11,6.488143e-12,4.589440e-12,1.018230e-11,3.958389e-12,0.0,1.532108e-14,2.944311e-13,5.890843e-13,4.611866e-13,1.998401e-15,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
6,2.012834e-12,1.517231e-12,1.012523e-12,7.860379e-13,7.043255e-13,0.0,2.546852e-13,5.510925e-12,1.139311e-11,5.809575e-12,7.014389e-13,2.950751e-12,7.895906e-13,5.337952e-13,3.486100e-13,2.362555e-13
7,3.201883e-13,1.137623e-11,1.702927e-11,4.202194e-12,1.534284e-11,0.0,1.243450e-14,1.110223e-14,5.604406e-13,6.894485e-13,3.841372e-14,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
8,3.653300e-12,7.641443e-12,1.661826e-11,6.361356e-12,1.138467e-11,0.0,8.881784e-15,2.218226e-13,2.404743e-13,2.109424e-13,1.021405e-14,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
9,2.773470e-11,3.025136e-11,2.086020e-11,3.302314e-11,2.768452e-11,0.0,5.329071e-15,3.952394e-14,4.278800e-13,5.064837e-13,6.883383e-15,3.019585e-12,7.178702e-13,1.936229e-13,4.596323e-14,1.243450e-14


Normalization:

In [21]:
spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums[spectrum_columns].sum(axis=1), axis=0)

The cell displays all information we have about spectrums now.

In [22]:
display(spectrums)

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,xmm_SC_EP_1_FLUX,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV
0,0.001615,0.005897,0.000000,0.006882,0.000827,0.0,8.589640e-03,0.151856,0.351404,0.339320,1.195570e-01,0.011464,0.002133,0.000387,0.000058,0.000010
1,0.012792,0.013542,0.000630,0.007092,0.026947,0.0,7.565786e-03,0.147922,0.319311,0.273791,6.634634e-02,0.024811,0.003781,0.001194,0.010990,0.083285
2,0.351958,0.175076,0.095055,0.264229,0.096284,0.0,2.420361e-04,0.006330,0.007298,0.003528,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.249396,0.145986,0.129537,0.204483,0.197701,0.0,1.135107e-03,0.017450,0.030100,0.024001,2.116300e-04,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.002762,0.006203,0.011182,0.006156,0.008600,0.0,9.787310e-03,0.193020,0.366693,0.329729,6.586824e-02,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.414902,0.142820,0.101025,0.224138,0.087134,0.0,3.372549e-04,0.006481,0.012967,0.010152,4.398977e-05,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.058239,0.043899,0.029296,0.022743,0.020379,0.0,7.369003e-03,0.159452,0.329646,0.168093,2.029527e-02,0.085376,0.022846,0.015445,0.010087,0.006836
7,0.006458,0.229440,0.343453,0.084751,0.309440,0.0,2.507837e-04,0.000224,0.011303,0.013905,7.747425e-04,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.078818,0.164859,0.358528,0.137242,0.245617,0.0,1.916186e-04,0.004786,0.005188,0.004551,2.203614e-04,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.191896,0.209309,0.144332,0.228487,0.191549,0.0,3.687185e-05,0.000273,0.002961,0.003504,4.762614e-05,0.020893,0.004967,0.001340,0.000318,0.000086


In [23]:
# spectrums.to_csv("markeddata/normalized_spectra.txt", index=False)

In [30]:
def normalize_spectra(catalog_name, s_spectrum_columns, gev_spectrum_columns, tev_spectrum_columns,
                      s_other_columns, gev_other_columns, tev_other_columns, log = True):
    other_columns = s_other_columns + gev_other_columns + tev_other_columns + ['marked']
    catalog = pd.read_csv("markeddata/" + catalog_name + ".txt", sep='\t', index_col=0)
    
    spectrum_columns =  s_spectrum_columns +gev_spectrum_columns + tev_spectrum_columns
    spectrums = catalog[spectrum_columns + other_columns].copy()
    spectrums = spectrums.fillna(0)
    if log:
        only_spectra = spectrums[spectrum_columns].copy()
        only_spectra = np.log(only_spectra)
        matrix_inf = (only_spectra == -np.inf)
        only_spectra[matrix_inf] = 0
        spectrums[spectrum_columns] = only_spectra
    spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums[spectrum_columns].sum(axis=1), axis=0)
    if log:
        only_spectra = spectrums[spectrum_columns].copy()
        only_spectra[matrix_inf] = - np.inf
        spectrums[spectrum_columns] = only_spectra
    spectrums = spectrums.fillna(0)
    add_name = ""
    if not log:
        add_name = "WITHOUT_LOG"
    spectrums.to_csv("markeddata/" + catalog_name + "_normalized" + add_name + ".txt", index=False, sep='\t')
    print("markeddata/" + catalog_name + "_normalized" + add_name + ".txt")

* adding xmm spectrum columns to the gevtevsimbad, tevsimbad and gevsimbad spectrums

* contain objects, which have spectrum information from catalogs (gev, tev, simbad) and are in their intersection, but were not found in xmm catalog

In [31]:
# cat_name = 'gevtevsimbadxmm_001'
cat_name = get_name_for_gevtevxmm()
# cat_name = 'gevtevsimbadxmm_012'

normalize_spectra(catalog_name=cat_name, 
                   s_spectrum_columns=xmm_spectrum_columns, 
                   gev_spectrum_columns=gev_spectrum_columns, 
                   tev_spectrum_columns=tev_spectrum_columns, 
                   s_other_columns=xmm_other_columns + ['s_MAIN_ID', 's_simbad', 's_class'],
                   gev_other_columns=gev_other_columns,
                   tev_other_columns=tev_other_columns,
                   log = False)

normalize_spectra(catalog_name=cat_name, 
                   s_spectrum_columns=xmm_spectrum_columns, 
                   gev_spectrum_columns=gev_spectrum_columns, 
                   tev_spectrum_columns=tev_spectrum_columns, 
                   s_other_columns=xmm_other_columns + ['s_MAIN_ID', 's_simbad', 's_class'],
                   gev_other_columns=gev_other_columns,
                   tev_other_columns=tev_other_columns,
                   log = True)




markeddata/gevtevxmm_simbadclasses_06_normalizedWITHOUT_LOG.txt
markeddata/gevtevxmm_simbadclasses_06_normalized.txt


/home/masha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()


In [32]:
cat_name = get_name_for_gevtev()
# cat_name = 'gevtevsimbadxmm_012'

normalize_spectra(catalog_name=cat_name, 
                   s_spectrum_columns=[], 
                   gev_spectrum_columns=gev_spectrum_columns, 
                   tev_spectrum_columns=tev_spectrum_columns, 
                   s_other_columns=[] + ['s_MAIN_ID', 's_simbad', 's_class'],
                   gev_other_columns=gev_other_columns,
                   tev_other_columns=tev_other_columns,
                   log = False)

normalize_spectra(catalog_name=cat_name, 
                   s_spectrum_columns=[], 
                   gev_spectrum_columns=gev_spectrum_columns, 
                   tev_spectrum_columns=tev_spectrum_columns, 
                   s_other_columns=[] + ['s_MAIN_ID', 's_simbad', 's_class'],
                   gev_other_columns=gev_other_columns,
                   tev_other_columns=tev_other_columns,
                   log = True)

/home/masha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()


markeddata/gevtev_simbadclasses_25_normalizedWITHOUT_LOG.txt
markeddata/gevtev_simbadclasses_25_normalized.txt


In [33]:
# calculate_spectra(catalog_name="gevtev_TeVspectrum", 
#                   s_spectrum_columns=[], 
#                   gev_spectrum_columns=gev_spectrum_columns, 
#                   tev_spectrum_columns=tev_spectrum_columns, 
#                   s_other_columns=[],
#                   gev_other_columns=gev_other_columns,
#                   tev_other_columns=tev_other_columns)
# calculate_spectra(catalog_name="tevsimbad_TeVspectrum", 
#                   s_spectrum_columns =s_spectrum_columns, 
#                   gev_spectrum_columns=[], 
#                   tev_spectrum_columns=tev_spectrum_columns, 
#                   s_other_columns=s_other_columns,
#                   gev_other_columns=[],
#                   tev_other_columns=tev_other_columns)
# calculate_spectra(catalog_name="gevsimbad", 
#                   s_spectrum_columns=s_spectrum_columns, 
#                   gev_spectrum_columns=gev_spectrum_columns, 
#                   tev_spectrum_columns=[], 
#                   s_other_columns=s_other_columns,
#                   gev_other_columns=gev_other_columns,
#                   tev_other_columns=[])